# Random Forest Classifier from scratch

## Why Random Forest?
`Decision tree` is susceptible to high variance if it is not pruned. 

`Bagging` is the combination of many Decision Tree models, and the prediction follows the crowd; but these tree are highly correlated.

`Random Forest`, the extension of `Bagging`, constrains the features that can be used to build the trees, forcing trees to be different. This, in turn, can give a lift in performance.

`Bagging` and `Random Forest` execute Decision Tree upon a sample with replacement of the training dataset. **Sample with replacement** means that the same row may be chosen and added to samples more than once.

Of that training sample, one-third of it is set aside as test data, known as the out-of-bag (`oob`) sample, used for cross-validation, finalizing that prediction.

**Key Benefits**
- Reduced risk of **overfitting**: Averaging of **uncorrelated trees** lowers the overall **variance** and prediction **error**
- Provides flexibility: Handle both **regression** and **classification** tasks with a high degree of accuracy
- Easy to determine **feature importance**: Gini importance, mean decrease in impurity (MDI) and permutation importance known as mean decrease accuracy (MDA)

**Key Challenges**
- Time-consuming process
- Requires more resources
- More complex

In [56]:
from csv import reader
from math import log,sqrt
from random import randrange, seed
from statistics import mode
import numpy as np
import time

In [35]:
dataset = [[2.771244718, 1.784783929, 0],
           [1.728571309, 1.169761413, 0],
           [3.678319846, 2.81281357, 0],
           [3.961043357, 2.61995032, 0],
           [2.999208922, 2.209014212, 0],
           [7.497545867, 3.162953546, 1],
           [9.00220326, 3.339047188, 1],
           [7.444542326, 0.476683375, 1],
           [10.12493903, 3.234550982, 1],
           [6.642287351, 3.319983761, 1]]

## Calculating splits

In [36]:
def gini_index(groups, classes):
	"""
	Gini_index(S,A) = Sum(|Si|/|S| * (1 - Sum(Pj*Pj))),
	where,
		A: Attribute A
		S: Root group
		Si: Child ith when apply attribute A
		Pj: % of class j
	Complexity: O(#groups x #classes)
	
	Args:
		`groups`: A list of groups after choose Attribute A
		`classes`: A list of classes' values
	Return:
		Gini_index(S,A)
	For example,
	>>> gini_index([[[1,0],[2,0]],[[0,1],[1,1],[2,1]]], [0,1])
	>>> 0.0
	"""
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Another way: calculate the entropy (slow)
def entropy(groups, classes):
	"""
	Entropy(S,A) = Sum(|Si|/|S| * E(Si))
	E(Si) = sum (-Pj log(Pj))
	"""
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	entropy = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			if p ==0 or p==1:
				score = 0
			else:
				score -= p * log(p)
		# weight the group score by its relative size
		entropy += score * (size / n_instances)
	return entropy

In [37]:
entropy([[[1,0],[2,0]],[[0,0],[1,1],[2,1]]], [0,1]), \
gini_index([[[1,0],[2,0]],[[0,0],[1,1],[2,1]]], [0,1])

(0.38190850097688767, 0.26666666666666666)

In [38]:
def test_split(index, value, dataset):
    """
    Split a dataset based on an attribute and an attribute value.
    Complexity: O(#rows)
    """
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right


def get_split(dataset, n_features):
    """
    This heuristic algorithm loops all rows (samples) and column (attributes), thus it is painful for computing.
    Complexity: O(#rows^2 x #columns) x O(gini_index)
    """
    classes = list(set(row[-1] for row in dataset))
    retAttribute, retValue, retGroups, min_gini = None, None, None, 999

    features = list()
    counter = 0
    while counter < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            counter = counter + 1
            features.append(index)
    for row in dataset:
        for attribute in features:
            groups = test_split(attribute, row[attribute], dataset)
            gini = gini_index(groups, classes)
            if gini < min_gini:
                min_gini = gini
                retAttribute, retValue, retGroups = attribute, row[attribute], groups
    return {'attribute': retAttribute, 'value': retValue, 'groups': retGroups}


get_split(dataset, 2)


{'attribute': 0,
 'value': 6.642287351,
 'groups': ([[2.771244718, 1.784783929, 0],
   [1.728571309, 1.169761413, 0],
   [3.678319846, 2.81281357, 0],
   [3.961043357, 2.61995032, 0],
   [2.999208922, 2.209014212, 0]],
  [[7.497545867, 3.162953546, 1],
   [9.00220326, 3.339047188, 1],
   [7.444542326, 0.476683375, 1],
   [10.12493903, 3.234550982, 1],
   [6.642287351, 3.319983761, 1]])}

In [39]:
def to_terminal(group):
    assert group, 'group should not be empty'
    outcomes = [row[-1] for row in group]
    return mode(outcomes)  # most frequency

def split(node, max_depth, min_size, depth, n_features):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:  # left or right is empty list
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, depth+1, n_features)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, depth+1, n_features)

## Build a tree

In [40]:
def build_tree(train_set, max_depth, min_size, n_features):
	root = get_split(dataset= train_set, n_features=n_features)
	split(root, max_depth, min_size, 0, n_features)
	return root

def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' %
              ((depth*'-', (node['attribute']+1), node['value'])))
        print_tree(node['left'], depth+2)
        print_tree(node['right'], depth+2)
    else:
        print('%s[%s]' % ((depth*'-', node)))  # This is cool!


tree = build_tree(dataset, 6, 1, 1)
print_tree(tree)
# print(tree)


[X2 < 3.163]
--[X1 < 7.445]
----[X2 < 1.785]
------[0]
------[X1 < 2.771]
--------[0]
--------[0]
----[1]
--[X1 < 7.498]
----[1]
----[X2 < 3.163]
------[1]
------[1]


In [41]:
def predict(tree, input_row):
	if input_row[tree['attribute']] < tree['value']:
		if isinstance(tree['left'], dict):
			return predict(tree['left'], input_row)
		else:
			return tree['left']
	else:
		if isinstance(tree['right'], dict):
			return predict(tree['right'], input_row)
		else:
			return tree['right']

## Build a forest

In [42]:
def subsample_idx(data_size, sample_size):
    """
    Retrieve from the `dataset` indices *randomly* with `sample_size`
    """
    sample_indices = list()
    counter = 0
    while(counter < sample_size):
        index = randrange(data_size)
        sample_indices.append(index)
        counter += 1
    return sample_indices

def bagging_predict(trees, row):
    return mode([predict(tree, row) for tree in trees])  # most freq

def random_forest(train_set, test_set, max_depth, min_size, n_features, n_trees, sample_size):
    trees = list()
    for i in range(n_trees):
        sample_indices = subsample_idx(len(train_set), sample_size)
        sample = [train_set[i] for i in sample_indices]
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test_set]
    return predictions

print(random_forest(dataset[:int(len(dataset)*7/10)], dataset[int(len(dataset)*7/10):], 5, 1, 1, 5, 4))

[0, 0, 0]


## Out-of-bag error

In [74]:
def f1_score(y, y_hat):
    score = np.mean(y == y_hat)*100
    return score

In [75]:
def oob_classifier_f1_score(train_set, max_depth, min_size, n_features, n_trees, sample_size):
    X_train = np.array([x[:-1] for x in train_set])
    y_train = np.array([x[-1] for x in train_set])
    n_samples = len(X_train)
    classes = list(set(y_train))
    n_classes = len(classes) # 2 for binary

    predictions = np.zeros((n_samples, n_classes))
    def encode(class_name):
        """
        Args: name of a class, can be str or int
        Return:
            one-hot encoding of that class
        For example,
        >>> list= ['a', 'b', 'c']
        >>> ecode('a'), ecode('b'), ecode('c')
        >>> ([1,0,0], [0,1,0], [0,0,1])
        """
        return [int(class_name == classes[i]) for i in range(n_classes)]
    
    for i in range(n_trees):
        # Sampling and collect out-of-bag(oob) indices
        sample_indices = subsample_idx(len(train_set), sample_size)
        oob_indices = np.array([i for i in range(n_samples) if i not in sample_indices])
        
        # Build a tree
        sample = [train_set[i] for i in sample_indices]
        tree = build_tree(sample, max_depth, min_size, n_features)
        
        # Accumulate probability over d classes in OOB dataset
        tree_preds = [encode(predict(tree, X_train[oob_index])) for oob_index in oob_indices]
        predictions[oob_indices] += tree_preds
    
    predicted_class_index = np.argmax(predictions, axis = 1)
    predicted_class = [classes[i] for i in predicted_class_index] # Decode to list of classes and find score
    return f1_score(y_train, predicted_class)

## Work with real data

In [44]:
# Load a CSV file
def load_csv(filename):
    file = open(filename, "rt")
    lines = reader(file)
    dataset = list(lines)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column])
        
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])  # concatenate lists of lists to a list
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            # test_set use to predict => no need to hold [class] data
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [73]:
seed(1)
# load and prepare data
filename = 'data/sonar.csv'
# filename = 'data/BankNote_Authentication.csv'
dataset = load_csv(filename)
# remove the string attributes
dataset.pop(0) 

# convert string attributes to float
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)

# Check if the last row is not INT{0,1}
if not isinstance(dataset[0][-1], int):
    print("The last row convert from str to int")
    str_column_to_int(dataset, len(dataset[0])-1)

n_folds = 5
max_depth = 5
min_size = 10
n_trees = [1, 5, 10]
n_features = round(sqrt(len(dataset[0]) - 1))
sample_size = len(dataset)
# (train_set, max_depth, min_size, n_features, n_trees, sample_size):
for n_tree in n_trees:
    print('# trees: %s' % n_tree)
    tic = time.perf_counter()
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, n_features, n_tree, sample_size)
    toc = time.perf_counter()
    print(f"======> Runned in {toc - tic:0.4f} seconds")
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    tic = time.perf_counter()
    oob_score = oob_classifier_f1_score(dataset, max_depth, min_size, n_features, n_tree, sample_size)
    toc = time.perf_counter()
    print(f"======> Runned in {toc - tic:0.4f} seconds")
    print('OOB_f1_score: %.3f%%' % oob_score)

The last row convert from str to int
# trees: 1
======> Runned in 0.7614 seconds
Mean Accuracy: 64.878%
======> Runned in 0.1531 seconds
OOB_f1_score: 51.691%
# trees: 5
======> Runned in 3.9699 seconds
Mean Accuracy: 74.634%
======> Runned in 0.7734 seconds
OOB_f1_score: 74.396%
# trees: 10
======> Runned in 7.7910 seconds
Mean Accuracy: 79.024%
======> Runned in 1.5726 seconds
OOB_f1_score: 75.362%


## Conclusion

Version 0.1:
* Accuracy:
    - 80.000% - 10 trees data/sonar.csv

`Note`: Increasing the number of tree upgrade the model's accuracy and also converge to around 80%.

Version 0.2: 11/09/2021
* Add Out-of-bag(**oob**) error utilizing unsample data for cross-validation (NOT **k cross-validation**), this test is particularly used in `Random Forest` algorithms.
* OOB_f1_score:
    - 74.038% - 10 trees data/sonar.csv
    
Version 0.3: 20/09/2021:
* Update **oob_f1_score** function, move **f1_score** outside of the function

## Refs
https://machinelearningmastery.com/implement-random-forest-scratch-python/

https://www.ibm.com/cloud/learn/random-forest

https://github.com/parrt/random-forest-importances/blob/master/src/rfpimp.py